In [1]:
#default_exp train3d_features

In [2]:
#export
from rsna_retro.imports import *
from rsna_retro.metadata import *
from rsna_retro.preprocess import *
from rsna_retro.train import *
from rsna_retro.train3d import *

In [3]:
torch.cuda.set_device(3)

In [4]:
#export
def get_3d_dsrc(df, path=path_jpg256):
    df_series = df.reset_index().set_index('SeriesInstanceUID').sort_values("ImagePositionPatient2")
    rct = ReadCT(df_series, path)
    sids = df_series.index.unique()[:100]
    splits = RandomSplitter(valid_pct=0.1, seed=42)(sids)
    dsrc = DataSource(sids, [[rct.x],[rct.y]], splits=splits)
    return dsrc

In [5]:
dsrc = get_3d_dsrc(df_any)

In [6]:
nrm = Normalize.from_stats(mean,std)
batch_tfms = L(nrm, Cuda(), IntToFloatTensor())

In [7]:
dbunch = DataBunch(
    TfmdDL(dsrc.train, bs=None, after_batch=batch_tfms, num_workers=8, shuffle=False),
    TfmdDL(dsrc.valid, bs=None, after_batch=batch_tfms, num_workers=8, shuffle=False)
)
dbunch.device = default_device()
dbunch.c = 6

In [8]:
x,y = dbunch.one_batch()
x.shape, y.shape

(torch.Size([46, 3, 256, 256]), torch.Size([46, 6]))

## Model

In [67]:
class DummyLoss:
    def __call__(self, p, *t, **kwargs): return torch.tensor(0, device=p.device).float()

In [68]:
learn = get_learner(dbunch, xresnet34, lf=DummyLoss(), metrics=[])

In [69]:
learn.load('runs/baseline-3')

In [70]:
learn.model = learn.model[0]
# learn.metrics = []

In [71]:
# learn.dbunch = get_test_data(df_tst, bs=1024, sz=224)
preds,targs = learn.get_preds(dl=dbunch.train_dl)

In [74]:
preds.shape

torch.Size([3277, 512, 1, 1])

In [72]:
for i,(xb,yb) in enumerate(dbunch.train_dl):
    learn.predict()
    learn.model(xb)
    break

TypeError: predict() missing 1 required positional argument: 'item'

In [13]:
dsrc.train.items

array(['ID_fd218f4d7b', 'ID_8f1d6547bb', 'ID_3a08a46ea1', ...,
       'ID_179e6236c3', 'ID_83c4e41ff6', 'ID_43e3ec0305'], dtype=object)

## Training

In [18]:
name = 'train3d_baseline'

In [19]:
do_fit(learn, 1, 4e-2, freeze=False)
learn.save(f'runs/{name}-1')

KeyboardInterrupt: 

In [ ]:
learn.dbunch = get_data(512, 224)
do_fit(learn, 12, 5e-3, freeze=False)
learn.save(f'runs/{name}-2')

## Submission

In [19]:
learn.load(f'runs/{name}-3')
sub_fn = f'subm/{name}'

In [ ]:
learn.dbunch = get_test_data(df_tst, bs=1024, sz=224)
preds,targs = learn.get_preds()

In [33]:
pred_csv = submission(df_tst, preds, fn=nm)
FileLink(f'{sub_fn}.csv')

In [36]:
api.competition_submit(f'{nm}.csv', '0.0744,rn34_repl (scl 0.88)', 'rsna-intracranial-hemorrhage-detection')

100%|██████████| 26.0M/26.0M [00:02<00:00, 9.52MB/s]


Successfully submitted to RSNA Intracranial Hemorrhage Detection

In [ ]:
api.competitions_submissions_list('rsna-intracranial-hemorrhage-detection')[0]

## Export

In [21]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 01_data_01_metadata_stage2.ipynb.
Converted 01_data_02_preprocess_windows.ipynb.
Converted 02_train_00_train.ipynb.
Converted 02_train_01_train.ipynb.
Converted 02_train_02_train_wgt.ipynb.
Converted 03_train3d_00_train3d.ipynb.
This cell doesn't have an export destination and was ignored:
e
This cell doesn't have an export destination and was ignored:
e
This cell doesn't have an export destination and was ignored:
e
Converted 03_train3d_01_3Dbatch.ipynb.
